1. Сравните метрики hit_rate@k, precision@k.   
Какую метрику использовать предпочтительно и почему?  
Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?
ОТВЕТ: Согласно информации из вебинара -
Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)
Precision@k = (#@k (первые k) рекомендованных товаров, которые являются релевантными) / (#@k рекомендованных товаров)
Соответственно, если в Hit Rate мы знаем был ли хотя бы 1 товар, т.е. просто факт попадания хотя бы одного товара и ничего больше, то в Precision мы уже можем видеть долю релевантных рекомендованных к общему количеству рекомендованных.
А вот примеры в которых более уместно использовать hit_rate достаточно непросто привести, т.к. мне кажется предпочтительнее использовать более показательные метрики. Но опять же, если задача порекомендовать пользователю хотя бы 1 релевантный товар, то можно использовать hit_rate. Например, это может быть выбор тюнинга, допустим, спойлер на багажник. Тут явно достаточно, что бы пользователю порекомендовали хотя бы 1 релевантный товар. Т.к. он больше 1го и не закажет.

2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.
ОТВЕТ:
На классическом примере функции потерь видно, что если брать ее производные в чистом виде, то они получаются достаточно громозкдие. При логарифмировани - все значительно упрощается. А критические точки остаются такими же.


3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

ОТВЕТ: AUC, в отличие от precision, значение AUC оценивает среднюю вероятность успеха персональных рекомендательных списков в более реальных условиях — когда списки просматривают живые люди, внимание которых распределено неравномерно. (c Хабра формулировку взял, честно признаюсь :) )


4. boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, MRR@8,  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

IDCG должен считаться для всех 8 позиций рекомендаций

**Пример выполнения 4го задания**

MSE на  
$y = [0, 2]$  
$y_{pred} = [1, 2]$

MSE = $0.5*((0 - 1)^2 + (2-2)^2) = 0.5$ 

In [59]:
import numpy as np
((np.array([0, 2]) - np.array([1, 2]))**2).mean()

0.5

In [60]:
import pandas as pd
boughted = [1, 3, 5, 7, 9, 11]
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

In [61]:
def precision_at_k(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [62]:
precision_at_k(recommended, boughted, k=8)

0.5

In [63]:
precision_at_k_h = (0 + 1 + 1 + 0 + 1 + 1 + 0 + 0) / (1 + 1 + 1 + 1 + 1 + 1 + 1 + 1)
precision_at_k_h

0.5

In [64]:
def recall_at_k(recommended_list, bought_list, k=8):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [65]:
recall_at_k(recommended, boughted, k=8)

0.6666666666666666

In [66]:
recall_at_k_h = (0 + 1 + 1 + 0 + 1 + 1 + 0 + 0)/ 6
recall_at_k_h

0.6666666666666666

In [67]:
def ap_k(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1): 
        print(i)
        
        if flags[i-1] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            print(p_k)
            sum_ += p_k
            
    result = sum_ / len(flags)
    
    return result

In [68]:
ap_k(recommended, boughted, k=8)

1
2
0.5
3
0.6666666666666666
4
5
0.6
6
0.6666666666666666
7
8


0.30416666666666664

In [69]:
ap_k_h = (1/8)*(0 + 1/2 + 2/3 + 0 + 3/5 + 4/6 + 0 + 0)
ap_k_h

0.30416666666666664

In [70]:
def dcg_at_k(r, k=8, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.

def ndcg_at_k(r, k=8, method=1):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [71]:
ndcg_at_k(recommended)

0.6511942571734283

In [72]:
import math
dcg_at_k_h = (1/8)*(0/1 + 1/math.log(2) + 1/math.log(3) + 0/math.log(4) + 1/math.log(5) + 1/math.log(6) + 0/math.log(7) + 0/math.log(8))
ideal_dcg_at_k_h = (1/8)*(1/1 + 1/math.log(2) + 1/math.log(3) + 1/math.log(4) + 1/math.log(5) + 1/math.log(6) + 1/math.log(7) + 1/math.log(8))
ndcg_at_k_h = dcg_at_k_h / ideal_dcg_at_k_h
ndcg_at_k_h

0.5653142737255068

In [73]:
def reciprocal_rank(recommended_list, bought_list, k=8):
    ranks=0.
    for i, item_rec in enumerate(recommended_list[:k]):
        for item_bought in bought_list:
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list[:k])

In [74]:
reciprocal_rank(recommended, boughted)

0.15

In [75]:
reciprocal_rank_h = (1/8)*(0/1 + 1/2 + 1/3 + 0/4 + 1/5 + 1/6 + 0/7 + 0/8)
reciprocal_rank_h

0.15